In [1]:
# SoilGrids LCC calculation algorithm
# to be run on 6 horizons : 0cm, 5cm, 15cm, 30cm, 60cm, 100cm
# run for each partition

# Import packages and dependencies
import numpy as np
import pandas as pd
import math
from simpledbf import Dbf5

from SoilGrid_LCC_functions import surf_texture, top_texture, awc1, awc2, awc3, awc4, awc5, erosion_risk_LCC, soil_tex_LCC, soil_tex_LCC, stoniness_LCC, perm_LCC, LAWC_LCC, lime_LCC, calc_LCC, calc_LCC_no_awc, limitation, limitation_noawc


In [2]:
# import datasets from all horizons
d01 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_0cm_dosso_slope.dbf"
d02 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_5cm_dosso.dbf"
d03 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_15cm_dosso.dbf"
d04 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_30m_dosso.dbf"
d05 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_60cm_dosso.dbf"
d06 = "C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_100cm_dosso.dbf"

# convert to pandas dataframes
dbf1 = Dbf5(d01)
dbf2 = Dbf5(d02)
dbf3 = Dbf5(d03)
dbf4 = Dbf5(d04)
dbf5 = Dbf5(d05)
dbf6 = Dbf5(d06)

df1 = dbf1.to_dataframe()
df2 = dbf2.to_dataframe()
df3 = dbf3.to_dataframe()
df4 = dbf4.to_dataframe()
df5 = dbf5.to_dataframe()
df6 = dbf6.to_dataframe()

In [3]:
# normalize texture percentages
df1['total_p'] = df1['sand'] + df1['silt'] + df1['clay']
df2['total_p'] = df2['sand'] + df2['silt'] + df2['clay']
df3['total_p'] = df3['sand'] + df3['silt'] + df3['clay']
df4['total_p'] = df4['sand'] + df4['silt'] + df4['clay']
df5['total_p'] = df5['sand'] + df5['silt'] + df5['clay']
df6['total_p'] = df6['sand'] + df6['silt'] + df6['clay']

df1['sand'] = df1['sand'] / df1['total_p']
df2['sand'] = df2['sand'] / df2['total_p']
df3['sand'] = df3['sand'] / df3['total_p']
df4['sand'] = df4['sand'] / df4['total_p']
df5['sand'] = df5['sand'] / df5['total_p']
df6['sand'] = df6['sand'] / df6['total_p']

df1['silt'] = df1['silt'] / df1['total_p']
df2['silt'] = df2['silt'] / df2['total_p']
df3['silt'] = df3['silt'] / df3['total_p']
df4['silt'] = df4['silt'] / df4['total_p']
df5['silt'] = df5['silt'] / df5['total_p']
df6['silt'] = df6['silt'] / df6['total_p']

df1['clay'] = df1['clay'] / df1['total_p']
df2['clay'] = df2['clay'] / df2['total_p']
df3['clay'] = df3['clay'] / df3['total_p']
df4['clay'] = df4['clay'] / df4['total_p']
df5['clay'] = df5['clay'] / df5['total_p']
df6['clay'] = df6['clay'] / df6['total_p']

In [4]:
    #rename columns so we can combine into one dataframe with all horizons

df1 = df1.rename(columns={'bulk_dens': "bulk_dens1", 'cec': 'cec1', 'clay': 'clay1', 'coarse_fra': 'coarse_fra1',
                                'silt': 'silt1', 'sand': 'sand1', 'ph': 'ph1', 'org_c_cont': 'org_c_cont1'
                                })

df2 = df2.rename(columns={'bulk_dens': "bulk_dens2", 'cec': 'cec2', 'clay': 'clay2', 'coarse_fra': 'coarse_fra2',
                                'silt': 'silt2', 'sand': 'sand2', 'ph': 'ph2', 'org_c_cont': 'org_c_cont2'
                                })

df3 = df3.rename(columns={'bulk_dens': "bulk_dens3", 'cec': 'cec3', 'clay': 'clay3', 'course_fra': 'coarse_fra3',
                                'silt': 'silt3', 'sand': 'sand3', 'ph': 'ph3', 'org_c_cont': 'org_c_cont3'
                                })

df4 = df4.rename(columns={'bulk_dens': "bulk_dens4", 'cec': 'cec4', 'clay': 'clay4', 'coarse_fra': 'coarse_fra4',
                                'silt': 'silt4', 'sand': 'sand4', 'ph': 'ph4', 'org_c': 'org_c_cont4'
                                })

df5 = df5.rename(columns={'bulk_dens': "bulk_dens5", 'cec': 'cec5', 'clay': 'clay5', 'course_fra': 'coarse_fra5',
                                'silt': 'silt5', 'sand': 'sand5', 'ph': 'ph5', 'org_c_cont': 'org_c_cont5'
                                })

df6 = df6.rename(columns={'bulk_dens': "bulk_dens6", 'cec': 'cec6', 'clay': 'clay6', 'coarse_fra': 'coarse_fra6',
                                'silt': 'silt6', 'sand': 'sand6', 'ph': 'ph6', 'org_c': 'org_c_cont6'
                                })

In [5]:
# merge all horizons into one dataframe
df0 = pd.concat([df1, df2, df3, df4, df5, df6], axis=1)

In [6]:
df0.columns

Index(['Join_Count', 'TARGET_FID', 'Join_Cou_1', 'TARGET_F_1', 'Join_Cou_2',
       'bulk_dens1', 'cec1', 'clay1', 'coarse_fra1', 'org_c_cont1', 'ph1',
       'prob_class', 'sand1', 'silt1', 'Id', 'gridcode', 'total_p',
       'Join_Count', 'TARGET_FID', 'bulk_dens2', 'cec2', 'clay2',
       'coarse_fra2', 'org_c_cont2', 'ph2', 'prob_class', 'sand2', 'silt2',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens3', 'cec3', 'clay3',
       'coarse_fra3', 'silt3', 'sand3', 'ph3', 'org_c_cont3', 'prob_class',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens4', 'cec4',
       'coarse_fra4', 'clay4', 'org_c_cont4', 'ph4', 'sand4', 'silt4',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens5', 'cec5', 'clay5',
       'coarse_fra5', 'silt5', 'sand5', 'ph5', 'org_c_cont5', 'prob_class',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens6', 'cec6',
       'coarse_fra6', 'clay6', 'org_c_cont6', 'ph6', 'sand6', 'silt6',
       'total_p'],
      dtype='object')

In [7]:
# calculate texture components over horizons by doing averages of profiles
# e.g. sand content for 0-5cm horizon = avergae of sand content for profile 0cm and profile 5cm
df0['sand_1'] = (df0['sand1'] + df0['sand2']) / 2
df0['sand_2'] = (df0['sand2'] + df0['sand3']) / 2
df0['sand_3'] = (df0['sand3'] + df0['sand4']) / 2
df0['sand_4'] = (df0['sand4'] + df0['sand5']) / 2
df0['sand_5'] = (df0['sand5'] + df0['sand6']) / 2
df0['silt_1'] = (df0['silt1'] + df0['silt2']) / 2
df0['silt_2'] = (df0['silt2'] + df0['silt3']) / 2
df0['silt_3'] = (df0['silt3'] + df0['silt4']) / 2
df0['silt_4'] = (df0['silt4'] + df0['silt5']) / 2
df0['silt_5'] = (df0['silt5'] + df0['silt6']) / 2
df0['clay_1'] = (df0['clay1'] + df0['clay2']) / 2
df0['clay_2'] = (df0['clay2'] + df0['clay3']) / 2
df0['clay_3'] = (df0['clay3'] + df0['clay4']) / 2
df0['clay_4'] = (df0['clay4'] + df0['clay5']) / 2
df0['clay_5'] = (df0['clay5'] + df0['clay6']) / 2
df0['coarse_fra_1'] = (df0['coarse_fra1'] + df0['coarse_fra2']) / 2
df0['coarse_fra_2'] = (df0['coarse_fra2'] + df0['coarse_fra3']) / 2
df0['coarse_fra_3'] = (df0['coarse_fra3'] + df0['coarse_fra4']) / 2
df0['coarse_fra_4'] = (df0['coarse_fra4'] + df0['coarse_fra5']) / 2
df0['coarse_fra_5'] = (df0['coarse_fra5'] + df0['coarse_fra6']) / 2
df0['ph_1'] = (df0['ph1'] + df0['ph2']) / 2
df0['ph_2'] = (df0['ph2'] + df0['ph3']) / 2
df0['ph_3'] = (df0['ph3'] + df0['ph4']) / 2
df0['ph_4'] = (df0['ph4'] + df0['ph5']) / 2
df0['ph_5'] = (df0['ph5'] + df0['ph6']) / 2
df0['org_c_cont_1'] = (df0['org_c_cont1'] + df0['org_c_cont2']) / 2
df0['org_c_cont_2'] = (df0['org_c_cont2'] + df0['org_c_cont3']) / 2
df0['org_c_cont_3'] = (df0['org_c_cont3'] + df0['org_c_cont4']) / 2
df0['org_c_cont_4'] = (df0['org_c_cont4'] + df0['org_c_cont5']) / 2
df0['org_c_cont_5'] = (df0['org_c_cont5'] + df0['org_c_cont6']) / 2

In [8]:
# drop old columns which are no longer useful
df0 = df0.drop(['bulk_dens1', 'cec1', 'clay1', 'coarse_fra1', 'org_c_cont1', 'ph1',
       'prob_class', 'sand1', 'silt1',
       'Join_Count', 'TARGET_FID', 'bulk_dens2', 'cec2', 'clay2',
       'coarse_fra2', 'org_c_cont2', 'ph2', 'prob_class', 'sand2', 'silt2',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens3', 'cec3', 'clay3',
       'coarse_fra3', 'silt3', 'sand3', 'ph3', 'org_c_cont3', 'prob_class',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens4', 'cec4',
       'coarse_fra4', 'clay4', 'org_c_cont4', 'ph4', 'sand4', 'silt4',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens5', 'cec5', 'clay5',
       'coarse_fra5', 'silt5', 'sand5', 'ph5', 'org_c_cont5', 'prob_class',
       'total_p', 'Join_Count', 'TARGET_FID', 'bulk_dens6', 'cec6',
       'coarse_fra6', 'clay6', 'org_c_cont6', 'ph6', 'sand6', 'silt6',
       'total_p'], axis = 1)

In [9]:
df0.columns

Index(['Join_Cou_1', 'TARGET_F_1', 'Join_Cou_2', 'Id', 'gridcode', 'sand_1',
       'sand_2', 'sand_3', 'sand_4', 'sand_5', 'silt_1', 'silt_2', 'silt_3',
       'silt_4', 'silt_5', 'clay_1', 'clay_2', 'clay_3', 'clay_4', 'clay_5',
       'coarse_fra_1', 'coarse_fra_2', 'coarse_fra_3', 'coarse_fra_4',
       'coarse_fra_5', 'ph_1', 'ph_2', 'ph_3', 'ph_4', 'ph_5', 'org_c_cont_1',
       'org_c_cont_2', 'org_c_cont_3', 'org_c_cont_4', 'org_c_cont_5'],
      dtype='object')

In [10]:
# apply functions
df0["surface_texture"] = df0.apply(surf_texture, axis=1)
df0["top_texture"] = df0.apply(top_texture, axis=1)

# calculate awc value for each horizon
df0["awc1"] = df0.apply(awc1, axis=1)
df0["awc2"] = df0.apply(awc2, axis=1)
df0["awc3"] = df0.apply(awc3, axis=1)
df0["awc4"] = df0.apply(awc4, axis=1)
df0["awc5"] = df0.apply(awc5, axis=1)
 # multiple awc value by depth
 # calculating AWC value by adding all of the horizons
df0["awc_total"] = df0["awc1"] * 5 + df0['awc2'] * 10 + df0['awc3'] * 15 + df0['awc4'] * 30 + df0['awc5'] * 40

# Conversions
# convert ph values
df0["ph_1"] = df0["ph_1"] / 10
df0["ph_2"] = df0["ph_2"] / 10
df0["ph_3"] = df0["ph_3"] / 10
df0["ph_4"] = df0["ph_4"] / 10
df0["ph_5"] = df0["ph_5"] / 10

# calculate slope
df0['slope'] = df0['gridcode'] / 10000

# calculate LCC for subclasses
df0["erosion_risk_LCC"] = df0.apply(erosion_risk_LCC, axis=1)
df0["soil_tex_LCC"] = df0.apply(soil_tex_LCC, axis=1)
df0["stoniness_LCC"] = df0.apply(stoniness_LCC, axis=1)
df0["perm_LCC"] = df0.apply(perm_LCC, axis=1)
df0["awc_LCC"] = df0.apply(LAWC_LCC, axis=1)
df0["lime_LCC"] = df0.apply(lime_LCC, axis=1)

df0["LCC_final"] = df0.apply(calc_LCC, axis=1)
df0["LCC_final_noawc"] = df0.apply(calc_LCC_no_awc, axis=1)
df0["limitation"] = df0.apply(limitation, axis=1)
df0["limitation_noawc"] = df0.apply(limitation_noawc, axis=1)

In [11]:
df0['limitation'].value_counts()

4s-a             17494
3s-r s-a          9181
3s-a              3818
3s-r s-a s-l      1409
5s-r               483
3s-a s-l            84
3e s-r s-a          29
4s-l                12
4e                   8
4e s-a               4
3e s-a               3
Name: limitation, dtype: int64

In [14]:
df0.head()

,Join_Cou_1,TARGET_F_1,Join_Cou_2,Id,gridcode,sand_1,sand_2,sand_3,sand_4,sand_5,...,erosion_risk_LCC,soil_tex_LCC,stoniness_LCC,perm_LCC,awc_LCC,lime_LCC,LCC_final,LCC_final_noawc,limitation,limitation_noawc
0,4,0,2,21377,30977,0.858586,0.831105,0.807853,0.790990,0.795000,...,1,2,1,1,4,1,4,2,4s-a,2s-t
1,4,1,2,27361,17745,0.741136,0.738687,0.695000,0.661634,0.658416,...,1,2,3,1,3,1,3,3,3s-r s-a,3s-r
2,4,2,5,21306,41638,0.877692,0.875693,0.840217,0.821289,0.833640,...,1,2,3,1,4,3,4,3,4s-a,3s-r s-l
3,1,3,2,18566,32129,0.812216,0.796105,0.748889,0.713535,0.708535,...,1,2,3,1,3,3,3,3,3s-r s-a s-l,3s-r s-l
4,4,4,4,27283,28042,0.779672,0.769977,0.744033,0.707895,0.706436,...,1,2,3,1,3,1,3,3,3s-r s-a,3s-r


In [15]:
df0_succinct = df0[["TARGET_F_1", "surface_texture", "top_texture", 'awc_total', "erosion_risk_LCC", 
                       "soil_tex_LCC", "stoniness_LCC", "perm_LCC", "awc_LCC", "lime_LCC", "LCC_final",
                       "LCC_final_noawc", "limitation", "limitation_noawc"]].copy()

In [16]:
df0_succinct.to_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\dosso_index\\SoilGrid_LCC_FIXED.csv")